### Import Statements

In [1]:
import os
import cv2
import pylab
import dlib
import numpy as np
import pandas as pd
#import imageio
import skimage.io as imageio
import sys
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
#from keras.utils import multi_gpu_model
from keras.optimizers import SGD, RMSprop
from tensorflow.keras.optimizers import Adam
from keras.layers import Concatenate, Input, concatenate, add, multiply, maximum, LSTM, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation
from sklearn import preprocessing
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
#K.set_image_dim_ordering('th')
#K.set_image_data_format('channels_first')
#import matplotlib
#matplotlib.use('Agg')

### GPU Check

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')

In [3]:
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
#K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

### Loading Excel to DataFrame

In [ ]:
df = pd.read_excel('/Thesis/Datasets/CASME2/CASME2-coding-20190701.xlsx') #index_col=0

In [ ]:
df.head()

In [ ]:
df['Subject'] = df['Subject'].astype(str)
df['Sub_FileName'] = df[['Subject', 'Filename']].apply(lambda x: '_'.join(x), axis=1)
df.head(5)

In [ ]:
positive = df[df['Estimated Emotion'] == 'happiness']['Sub_FileName']#[['Subject','Filename']]
print('Positive :', positive.count())

negative = df[(df['Estimated Emotion'] == 'disgust') | (df['Estimated Emotion'] == 'repression') | (df['Estimated Emotion'] == 'fear') | (df['Estimated Emotion'] == 'sadness')]['Sub_FileName']
print('Negative :',negative.count())

surprise = df[df['Estimated Emotion'] == 'surprise']['Sub_FileName']
print('Surprise :',surprise.count())

### Initialization

In [ ]:
subjectPath = '/Thesis/Datasets/CASME2/CASME2_Cropped_updated/Cropped/'

In [ ]:
image_rows, image_columns, frames_Count = 64, 64, 48
training_casme_list = []

### SAMM Dataset

In [ ]:
sub_count = 0
for lab in [positive, negative, surprise]: 
    subDirectory = lab#.tolist() 
    count = 0
    for sub in tqdm(subDirectory):
        sub = str(sub).split('_')
        labelpath = '/Thesis/Datasets/CASME2/CASME2_RAW_selected/'+'sub'+ sub[0].zfill(2) +'/'
        directorylisting = os.listdir(labelpath)
        for video in directorylisting:
            if video == str(sub[1]+'_'+sub[2]):
                frames = []
                total_face_frames = []
                videopath = labelpath + video
                framelisting = os.listdir(videopath)
                if len(framelisting) > frames_Count:
                    framerange = [x  for x in range(frames_Count)]
                    count = count + 1
                    for frame in framerange:
                        imagepath = videopath + "/" + framelisting[frame]
                        image = cv2.imread(imagepath)
                        imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                        grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                        frames.append(grayimage)
                    frames = np.asarray(frames)
                    #videoarray = np.rollaxis(np.rollaxis(frames, 2, 0), 2, 0)
                    training_casme_list.append(frames)
                # break
    
    print(count)
    # break

In [ ]:
len(training_casme_list)

In [ ]:
training_casme_list[2].shape

In [ ]:
290+744

### Creating Target labels

In [ ]:
training_casme_list = np.asarray(training_casme_list)
training_casme_samples = len(training_casme_list)

training_casme_labels = np.zeros((training_casme_samples, ), dtype = int)

training_casme_labels[0:27] = 0
training_casme_labels[27:103] = 1
training_casme_labels[103:124] = 2

training_casme_labels = np_utils.to_categorical(training_casme_labels, 3)

In [ ]:
training_casme_data = [training_casme_list, training_casme_labels]
(training_frames_casme, traininglabels_casme) = (training_casme_data[0], training_casme_data[1])
training_casme_set = np.zeros((training_casme_samples, 1, image_rows, image_columns, frames_Count))
for h in range(training_casme_samples):
    training_casme_set[h][0][:][:][:] = training_frames_casme[h,:,:,:]

In [ ]:
training_casme_set = training_casme_set.astype('float32')
training_casme_set -= np.mean(training_casme_set)
training_casme_set /= np.max(training_casme_set)

In [ ]:
training_casme_set.shape

#### Save Numpy Arrays

In [ ]:
# Save Nump Arrays to save time
np.save('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_images.npy', training_casme_set)
np.save('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_labels.npy', training_casme_labels)

#### Loading Numpy arrays

In [ ]:
training_casme_set = None
training_casme_labels = None
training_casme_set  = np.load('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_images.npy')
training_casme_labels = np.load('/Thesis/Multi_Scale_MicroExp/Sample_datasets/microexp_casme_labels.npy')

training_casme_set.shape

### Model

In [ ]:
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(1, image_rows, image_columns, frames_Count), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

#model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

model.summary()

In [ ]:
# Load pre-trained weights

#model.load_weights('/Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn/weights-improvement-53-0.88.hdf5')

#model.load_weights('/Thesis/MicroExpSTCNN/CASME_SQUARE/weights-improvement-53-0.88.hdf5')

#### Model Compile

In [ ]:
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=10, decay_rate=0.90, staircase=True)

model.compile( loss="categorical_crossentropy", optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])

#### Creating checkpoints

In [ ]:
filepath="/Thesis/Multi_Scale_MicroExp/Sample_datasets/weights_microexp/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor="val_accuracy", patience=25)

callbacks_list = [checkpoint, early_stopping]

#### Spliting the dataset into training and validation sets

In [ ]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_casme_set, training_casme_labels, test_size=0.2, random_state=4)

#### Save validation set in a numpy array

In [ ]:
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_images.npy', validation_images)
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_labels.npy', validation_labels)

#### Load validation set from numpy array

In [ ]:
validation_images = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_images.npy')
validation_labels = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_labels.npy')

#### Pre-Trained data

In [ ]:
#validation_images = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_images.npy')
#validation_labels = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_labels.npy')

#### Training the model

In [ ]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), callbacks=callbacks_list, batch_size = 8, epochs = 100, shuffle=True)


#### Finding Confusion Matrix

In [ ]:
predictions = model.predict(validation_images)
predictions_labels = np.argmax(predictions, axis=1)
validation_labels = np.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)